# Molecular Docking example

Most of this code comes from [here](https://chem-workflows.com/articles/2021/09/18/1-molecular-docking/?fbclid=IwAR0CgvxmSlGgBsUuTX7nHh4r4SryAA9I9PeYovPrHk7te0jTWDFwxVq-snA)

In [2]:
SDF_FILE = '../molecules/human_serum_albumin_CHEMBL_data.sdf'
PDB_FILE = '../molecules/4la0_human_serum_albumin_cleaned_without_ligand.pdb'

## The necessary imports

In [3]:
import sys
sys.path.append('..')

from sbap import pipeline

In [4]:
from pymol import cmd
import py3Dmol

from vina import Vina

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

from meeko import MoleculePreparation
from meeko import obutils

import sys, os
sys.path.insert(1, '../external')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork

import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

We will place all our temporary files in the 'working' directory not tracked by git

In [10]:
if not os.path.isdir('../working'):
    os.mkdir('../working')
os.chdir('../working')

### Ligand preparation

In [11]:
mol= [m for m in pybel.readfile(filename=SDF_FILE,format='sdf')][148]  # just a random one
mol.addh()
out=pybel.Outputfile(filename='ligandH.mol2',format='mol2',overwrite=True)
out.write(mol)
out.close()

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warnin

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file

*** Open Babel Warnin

First problem - why are there those errors? How big a difference does it make?

In [14]:
! python3 ../external/mk_prepare_ligand.py -i ligandH.mol2 -o ligandH.pdbqt

An alternative - I didn't get it to work

In [13]:
#sdf_file = Chem.SDMolSupplier(SDF_FILE)
#sample_ligand = sdf_file[248]
#ligand = Chem.AddHs(sample_ligand)
#AllChem.EmbedMolecule(ligand, AllChem.ETKDG())
#writer = Chem.rdmolfiles.SDWriter('../molecules/ligandH.sdf')
#writer.write(ligand)
! #python3 ../external/mk_prepare_ligand.py -i ../molecules/ligandH.sdf -o ../molecules/ligand.pdbqt

### Protein (receptor) preparation

In [34]:
fix_protein(filename=PDB_FILE, addHs_pH=7.4, try_renumberResidues=True, output='proteinH.pdb')

There is a mysterious parameter:
- addHs_pH: float ; Add hydrogens at user defined pH

I copied the value which was in their code, have no idea how to choose the correct pH...

In [35]:
! obabel -ipdb proteinH.pdb -opdbqt -h

REMARK  Name = proteinH.pdb
REMARK  2044 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: N_1  and  CA_5
REMARK    2  A    between atoms: CA_5  and  C_7
REMARK    3  A    between atoms: CA_5  and  CB_9
REMARK    4  A    between atoms: N_13  and  CA_15
REMARK    5  A    between atoms: CA_15  and  C_17
REMARK    6  A    between atoms: CA_15  and  CB_19
REMARK    7  A    between atoms: CB_19  and  CG_22
REMARK    8  A    between atoms: N_30  and  CA_32
REMARK    9  A    between atoms: CA_32  and  C_34
REMARK   10  A    between atoms: CA_32  and  CB_36
REMARK   11  A    between atoms: CB_36  and  CG_39
REMARK   12  A    between atoms: CG_39  and  CD_42
REMARK   13  A    between atoms: CD_42  and  CE_45
REMARK   14  A    between atoms: CE_45  and  NZ_48
REMARK   15  A    between atoms: N_52  and  CA_54
REMARK   16  A    between atoms: CA_54  and  C_56
REMARK   17  A    between atoms: CA_54  and  CB_58
REMARK   18  A    between atoms: CB_58

ROOT
ATOM      1  CA  ILE A 290      12.524 -11.100   4.652  0.00  0.00    +0.000 C 
ENDROOT
BRANCH   1   2
ATOM      2  C   ILE A 290      12.042  -9.752   4.063  0.00  0.00    +0.000 C 
ATOM      3  O   ILE A 290      12.826  -8.811   3.886  0.00  0.00    +0.000 OA
ATOM      4  N   ALA A 291      10.745  -9.653   3.780  0.00  0.00    +0.000 N 
ATOM      5  H   ALA A 291       9.909 -10.470   3.917  0.00  0.00    +0.000 HD
BRANCH   4   6
ATOM      6  CA  ALA A 291      10.184  -8.421   3.224  0.00  0.00    +0.000 C 
ATOM      7  CB  ALA A 291       8.784  -8.659   2.687  0.00  0.00    +0.000 C 
BRANCH   6   8
ATOM      8  C   ALA A 291      10.185  -7.250   4.217  0.00  0.00    +0.000 C 
ATOM      9  O   ALA A 291      10.290  -6.101   3.795  0.00  0.00    +0.000 OA
ATOM     10  N   GLU A 292      10.081  -7.514   5.521  0.00  0.00    +0.000 N 
ATOM     11  H   GLU A 292       9.994  -8.510   6.149  0.00  0.00    +0.000 HD
BRANCH  10  12
ATOM     12  CA  GLU A 292      10.005  -6.414 

BRANCH15361538
ATOM   1538  CA  GLN A 580     -34.214  28.006   6.202  0.00  0.00    +0.000 C 
BRANCH15381539
ATOM   1539  C   GLN A 580     -34.380  29.516   6.403  0.00  0.00    +0.000 C 
ATOM   1540  O   GLN A 580     -34.988  30.211   5.577  0.00  0.00    +0.000 OA
ATOM   1541  N   ALA A 581     -33.800  30.022   7.488  0.00  0.00    +0.000 N 
ATOM   1542  H   ALA A 581     -33.262  29.467   8.379  0.00  0.00    +0.000 HD
BRANCH15411543
ATOM   1543  CA  ALA A 581     -33.875  31.440   7.799  0.00  0.00    +0.000 C 
ATOM   1544  CB  ALA A 581     -33.494  31.675   9.260  0.00  0.00    +0.000 C 
BRANCH15431545
ATOM   1545  C   ALA A 581     -32.993  32.285   6.864  0.00  0.00    +0.000 C 
ATOM   1546  O   ALA A 581     -33.379  33.396   6.493  0.00  0.00    +0.000 OA
BRANCH15451547
ATOM   1547  N   ALA A 582     -31.845  31.758   6.439  0.00  0.00    +0.000 NA
ATOM   1548  H   ALA A 582     -31.228  30.972   7.074  0.00  0.00    +0.000 HD
ATOM   1549  CA  ALA A 582     -30.960  32.52

BRANCH19711972
ATOM   1972  OG  SER A 489       2.906  19.230  -0.183  0.00  0.00    +0.000 OA
ATOM   1973  HG  SER A 489       3.636  19.573   0.673  0.00  0.00    +0.000 HD
ENDBRANCH19711972
ENDBRANCH10591971
ENDBRANCH10571059
ENDBRANCH10541055
BRANCH10541974
ATOM   1974  CB  PHE A 488      -0.481  16.748  -3.340  0.00  0.00    +0.000 C 
BRANCH19741975
ATOM   1975  CG  PHE A 488      -1.348  15.811  -4.127  0.00  0.00    +0.000 A 
ATOM   1976  CD1 PHE A 488      -2.713  16.051  -4.250  0.00  0.00    +0.000 A 
ATOM   1977  CD2 PHE A 488      -0.803  14.694  -4.751  0.00  0.00    +0.000 A 
ATOM   1978  CE1 PHE A 488      -3.518  15.195  -4.986  0.00  0.00    +0.000 A 
ATOM   1979  CE2 PHE A 488      -1.602  13.835  -5.499  0.00  0.00    +0.000 A 
ATOM   1980  CZ  PHE A 488      -2.955  14.088  -5.616  0.00  0.00    +0.000 A 
ENDBRANCH19741975
ENDBRANCH10541974
ENDBRANCH10521054
ENDBRANCH10471050
BRANCH 9141981
ATOM   1981  CB  VAL A 462      -6.148   4.151  -7.160  0.00  0.00    +0.000

ATOM   2477  CB  LYS A 378      21.996  21.236  -0.804  0.00  0.00    +0.000 C 
BRANCH24772478
ATOM   2478  CG  LYS A 378      21.718  21.990   0.503  0.00  0.00    +0.000 C 
BRANCH24782479
ATOM   2479  CD  LYS A 378      22.696  23.135   0.790  0.00  0.00    +0.000 C 
BRANCH24792480
ATOM   2480  CE  LYS A 378      24.065  22.591   1.216  0.00  0.00    +0.000 C 
BRANCH24802481
ATOM   2481  NZ  LYS A 378      25.094  23.656   1.459  0.00  0.00    +0.000 N 
ATOM   2482  HZ1 LYS A 378      26.210  23.230   1.604  0.00  0.00    +0.000 HD
ATOM   2483  HZ2 LYS A 378      25.219  24.444   0.561  0.00  0.00    +0.000 HD
ATOM   2484  HZ3 LYS A 378      24.846  24.329   2.424  0.00  0.00    +0.000 HD
ENDBRANCH24802481
ENDBRANCH24792480
ENDBRANCH24782479
ENDBRANCH24772478
ENDBRANCH 4692477
ENDBRANCH 467 469
ENDBRANCH 464 465
BRANCH 4642485
ATOM   2485  CB  PHE A 377      21.643  16.583  -3.827  0.00  0.00    +0.000 C 
BRANCH24852486
ATOM   2486  CG  PHE A 377      22.014  15.755  -5.048  0.00  0.

BRANCH34123416
ATOM   3416  CA  ASP A 187      -4.531  -2.349  10.594  0.00  0.00    +0.000 C 
BRANCH34163419
ATOM   3417  C   ARG A 186      -6.077  -4.164   9.989  0.00  0.00    +0.000 C 
ATOM   3418  O   ARG A 186      -6.046  -3.865   8.798  0.00  0.00    +0.000 OA
ATOM   3419  N   ASP A 187      -5.339  -3.543  10.914  0.00  0.00    +0.000 N 
ATOM   3420  H   ASP A 187      -5.466  -3.537  12.086  0.00  0.00    +0.000 HD
BRANCH34173421
ATOM   3421  CA  ARG A 186      -6.934  -5.350  10.443  0.00  0.00    +0.000 C 
BRANCH34213424
ATOM   3422  C   LEU A 185      -5.123  -6.884  10.795  0.00  0.00    +0.000 C 
ATOM   3423  O   LEU A 185      -4.796  -6.811   9.611  0.00  0.00    +0.000 OA
ATOM   3424  N   ARG A 186      -6.189  -6.268  11.276  0.00  0.00    +0.000 N 
ATOM   3425  H   ARG A 186      -6.320  -5.977  12.408  0.00  0.00    +0.000 HD
BRANCH34223426
ATOM   3426  CA  LEU A 185      -4.333  -7.741  11.786  0.00  0.00    +0.000 C 
BRANCH34263429
ATOM   3427  C   GLU A 184    

BRANCH44654467
ATOM   4467  OD1 ASP A  13       6.792 -27.008  -8.781  0.00  0.00    +0.000 OA
ATOM   4468  H   ASP A  13       6.308 -27.624  -8.267  0.00  0.00    +0.000 HD
ENDBRANCH44654467
ENDBRANCH44644465
ENDBRANCH43414464
ENDBRANCH43374341
ENDBRANCH43364339
BRANCH43364469
ATOM   4469  CB  LEU A  14       1.485 -23.917  -5.000  0.00  0.00    +0.000 C 
BRANCH44694470
ATOM   4470  CG  LEU A  14       2.017 -22.768  -5.855  0.00  0.00    +0.000 C 
ATOM   4471  CD1 LEU A  14       0.912 -22.180  -6.702  0.00  0.00    +0.000 C 
ATOM   4472  CD2 LEU A  14       2.642 -21.730  -4.948  0.00  0.00    +0.000 C 
ENDBRANCH44694470
ENDBRANCH43364469
ENDBRANCH43324336
ENDBRANCH43314334
ENDBRANCH43274331
ENDBRANCH43264329
BRANCH43264473
ATOM   4473  CB  GLU A  16      -3.704 -29.571  -2.695  0.00  0.00    +0.000 C 
BRANCH44734474
ATOM   4474  CG  GLU A  16      -5.228 -29.446  -2.816  0.00  0.00    +0.000 C 
BRANCH44744475
ATOM   4475  CD  GLU A  16      -5.970 -30.776  -2.758  0.00  0.00    +0

ATOM   4946  OE2 GLU A 119     -22.273  -9.475  21.529  0.00  0.00    +0.000 OA
ATOM   4947  H   GLU A 119     -22.978 -10.015  21.828  0.00  0.00    +0.000 HD
ENDBRANCH49444946
ENDBRANCH49434944
ENDBRANCH49424943
ENDBRANCH37804942
ENDBRANCH37763780
ENDBRANCH37753778
BRANCH37754948
ATOM   4948  CB  VAL A 120     -15.374 -12.919  20.967  0.00  0.00    +0.000 C 
ATOM   4949  CG1 VAL A 120     -14.314 -13.962  21.241  0.00  0.00    +0.000 C 
ATOM   4950  CG2 VAL A 120     -14.750 -11.559  20.656  0.00  0.00    +0.000 C 
ENDBRANCH37754948
ENDBRANCH37713775
ENDBRANCH37703773
BRANCH37704951
ATOM   4951  CB  ASP A 121     -19.384 -16.134  22.294  0.00  0.00    +0.000 C 
BRANCH49514952
ATOM   4952  CG  ASP A 121     -19.385 -17.361  23.206  0.00  0.00    +0.000 C 
ATOM   4953  OD1 ASP A 121     -18.535 -18.262  22.996  0.00  0.00    +0.000 OA
BRANCH49524954
ATOM   4954  OD2 ASP A 121     -20.196 -17.400  24.160  0.00  0.00    +0.000 OA
ATOM   4955  H   ASP A 121     -20.813 -16.705  24.279  0.

BRANCH54265427
ATOM   5427  CG  ASP A 249       0.091 -13.726 -14.755  0.00  0.00    +0.000 C 
ATOM   5428  OD1 ASP A 249      -0.969 -14.078 -15.321  0.00  0.00    +0.000 OA
BRANCH54275429
ATOM   5429  OD2 ASP A 249       0.920 -12.940 -15.288  0.00  0.00    +0.000 OA
ATOM   5430  H   ASP A 249       1.702 -12.702 -14.829  0.00  0.00    +0.000 HD
ENDBRANCH54275429
ENDBRANCH54265427
ENDBRANCH33275426
BRANCH33175431
ATOM   5431  CB  HIS A 247       0.019  -9.127 -14.793  0.00  0.00    +0.000 C 
BRANCH54315432
ATOM   5432  CG  HIS A 247      -0.478  -9.310 -16.199  0.00  0.00    +0.000 A 
ATOM   5433  CD2 HIS A 247      -0.966  -8.418 -17.099  0.00  0.00    +0.000 A 
ATOM   5434  ND1 HIS A 247      -0.466 -10.534 -16.842  0.00  0.00    +0.000 N 
ATOM   5435  HD1 HIS A 247       0.310 -11.425 -16.985  0.00  0.00    +0.000 HD
ATOM   5436  CE1 HIS A 247      -0.946 -10.390 -18.067  0.00  0.00    +0.000 A 
ATOM   5437  NE2 HIS A 247      -1.244  -9.114 -18.251  0.00  0.00    +0.000 NA
ENDBRA

The [ADFR Suite](https://ccsb.scripps.edu/adfr/) helper programs looked promising, but I had this error 'cannot find reduce_wwPDB_het_dict.txt', no idea what this file does, but it is HUGE, like 1.3milion lines

In [ ]:
! #/home/justyna/ADFRsuite-1.0/bin/reduce {PDB_FILE} > proteinH.pdb
! #/home/justyna/ADFRsuite-1.0/bin/prepare_receptor -r proteinH.pdb -o protein.pdbqt

## Docking
- Step 1. calculate box
- Step 2. perform docking using Vina

In [36]:
cmd.load(filename='proteinH.pdb',format='pdb',object='prot')
cmd.load(filename='ligandH.mol2',format='mol2',object='lig')

center, size= getbox(selection='lig',extending=5.0,software='vina')

cmd.delete('all')

print(center,'\n',size)

{'center_x': 5.259149968624115, 'center_y': 1.2372499704360962, 'center_z': 0.0} 
 {'size_x': 19.122099995613098, 'size_y': 20.0106999874115, 'size_z': 10.0}


In [39]:
v = Vina(sf_name='vina')

v.set_receptor('proteinH.pdbqt')

v.set_ligand_from_file('ligand.pdbqt')

v.compute_vina_maps(center=[center['center_x'], center['center_y'], center['center_z']], 
                    box_size=[size['size_x'], size['size_y'], size['size_z']])

# Dock the ligand
v.dock(exhaustiveness=10, n_poses=10)
v.write_poses('docking_output.pdbqt', n_poses=10, overwrite=True)


mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1            0          0          0
   2     0.002505      6.092      8.231
   3     0.006104      5.884      7.105
   4      0.01438      5.096      6.782
   5      0.01575      7.313      9.288
   6      0.01649      9.259      11.13
   7      0.02939      4.611      8.093
   8      0.03169      4.192      6.326
   9      0.03199      8.809      11.73
  10       0.0379      6.124      7.444
Computing Vina grid ... done.
Performing docking (random seed: 1492869852) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


## Visualisation

In [42]:
pdbqt_to_sdf(pdbqt_file='docking_output.pdbqt', output='docking_output.sdf')
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('proteinH.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

view.addModel(open('ligandH.mol2','r').read(),format='sdf')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('docking_output.sdf')

p=Chem.MolToMolBlock(results[0],False)

print('Reference: Magenta | Vina Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

Reference: Magenta | Vina Pose: Cyan
Pose: 1 | Score: 0.000


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is docking_output.

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Attempt at calculating interaction fingerprint

In [43]:
# load protein
prot = mda.Universe("proteinH.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
lig_suppl = list(plf.sdf_supplier('docking_output.sdf'))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

  0%|          | 0/10 [00:00<?, ?it/s]

ligand               UNL1                                            \
protein          ALA194.A      ASP451.A      GLN459.A      LEU198.A   
interaction    VdWContact    VdWContact    VdWContact    VdWContact   
Frame                                                                 
0            (None, None)  (None, None)  (None, None)        (0, 0)   
1            (None, None)  (None, None)  (None, None)  (None, None)   
2            (None, None)       (10, 0)  (None, None)  (None, None)   
3            (None, None)  (None, None)  (None, None)        (8, 0)   
4            (None, None)  (None, None)  (None, None)        (0, 0)   
5            (None, None)  (None, None)  (None, None)       (34, 0)   
6                 (25, 0)  (None, None)  (None, None)        (8, 0)   
7            (None, None)  (None, None)  (None, None)  (None, None)   
8            (None, None)  (None, None)       (31, 0)  (None, None)   
9            (None, None)       (33, 0)  (None, None)        (0, 0)   

ligand                                                               \
protein          LYS195.A      LYS199.A      SER454.A      TRP214.A   
interaction    VdWContact    VdWContact    VdWContact    VdWContact   
Frame                                                                 
0            (None, None)       (25, 0)  (None, None)        (0, 0)   
1            (None, None)       (10, 0)  (None, None)        (0, 0)   
2            (None, None)  (None, None)       (24, 0)  (None, None)   
3            (None, None)  (None, None)  (None, None)  (None, None)   
4            (None, None)  (None, None)  (None, None)        (8, 0)   
5                 (19, 0)       (24, 0)  (None, None)  (None, None)   
6                 (11, 0)  (None, None)  (None, None)  (None, None)   
7            (None, None)        (7, 0)  (None, None)  (None, None)   
8                 (38, 0)  (None, None)  (None, None)  (None, None)   
9            (None, None)  (None, None)  (None, None)  (None, None)   

ligand                     
protein          VAL455.A  
interaction    VdWContact  
Frame                      
0                 (12, 0)  
1            (None, None)  
2            (None, None)  
3                  (5, 0)  
4                  (1, 0)  
5                  (1, 0)  
6                  (0, 0)  
7                 (36, 0)  
8                  (3, 0)  
9                  (0, 0)